<a href="https://colab.research.google.com/github/sreedeepack/Q-Exchange/blob/dev/IR%20workbench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import reprlib
from collections import defaultdict
from indexing import Preprocessor,Indexer
import re

In [3]:
# for testing
doc_map = defaultdict()

In [4]:
import jsonlines

def clean_str(text) :
        text = (text.encode('ascii', 'ignore')).decode("utf-8")
        text = re.sub("&.*?;", "", text)
        text = re.sub("[\]\|\[\@\,\$\%\*\&\\\(\)\":]", "", text)
        text = re.sub("-", " ", text)
        text = re.sub("\.+", "", text)
        text = re.sub("^\s+","" ,text)
        text = re.sub("\.+", "", text)
        text = text.lower()
        return text

def document_generator(file):
    with jsonlines.open(file) as reader:
        for doc_id, obj in enumerate(reader):
            item = {'doc_id': doc_id, 'url': obj['url'], 'title': clean_str(obj['title']), 'desc': clean_str(obj['desc'])}
            yield item

In [5]:
p = Preprocessor()
indexer = Indexer(p)

In [6]:
inverted = {}
for doc in document_generator("crawlers/stack/data/solr.jsonl"):
    doc_id = doc['doc_id']
    # index both title and description
    text = doc['title'] + " " +doc['desc']
    doc_map[doc_id] = (doc['url'], text)
    doc_index = indexer.inverted_index(text)
    indexer.inverted_index_add(inverted,doc_id=doc_id,doc_index=doc_index)

In [7]:
# Print Inverted-Index (3 rows)
i = 0
for word, doc_locations in inverted.items():
    print(word, reprlib.repr(doc_locations))

    i += 1
    if i > 3:
        break

instal {0: [7, 72, 130, 261, 532, 816, ...], 115: [1911, 2734], 188: [831, 1578], 829: [6270, 11787], ...}
uniti {0: [15, 80, 99, 115, 178, 380, ...], 158: [5019], 365: [7622], 473: [3916], ...}
use {0: [28, 1049, 1343, 1611, 1759, 1880, ...], 1: [1041], 2: [112, 241, 377, 776, 812, 823, ...], 3: [1692, 3680, 3940], ...}
ubuntu {0: [34, 599, 624, 944, 1254], 1408: [1420, 3081], 1444: [2089, 4014, 6279], 1476: [5540, 7515], ...}


Sample querying

In [10]:
# Search something and print results
queries = ['dolby', 'vim emacs', 'github week']

for query in queries:

    tokenized_query = ' '.join(p.tokenize_string(query))
    result_docs = indexer.search(inverted, tokenized_query)
    print(f"Search for '{query}': doc_ids={result_docs}")

    for _, word in p.word_index(tokenized_query):

        def extract_text(doc_id, position):
            return doc_map[doc_id][1][position:position+50].replace("\n", ' ')

        for doc_id in result_docs:
            for position in inverted[word][doc_id]:
                print(
                    f"\t - {extract_text(doc_id, position)}..."
                    f"\n\t -->{doc_map[doc_id][0]}"
                    f"\n"
                )
        print("\n")

Search for 'dolby': doc_ids={11074}
	 - dolby digital expires at midnight ...
	 -->/r/programming/comments/60b7kv/the_last_patent_on_ac3_dolby_digital_expires_at/



Search for 'vim emacs': doc_ids={3942, 2058, 2672, 3315, 3732, 4221, 1431, 2587, 2877}
	 - vim running inside gnome terminal showing the diff...
	 -->https://askubuntu.com/questions/48299/what-ides-are-available-for-ubuntu

	 - vim splits or extra tabs you like to install it in...
	 -->https://askubuntu.com/questions/48299/what-ides-are-available-for-ubuntu

	 - vim emacs nano gedit kate to name a few enable res...
	 -->https://askubuntu.com/questions/68918/how-do-i-restrict-my-kids-computing-time

	 - vim or emacs to write c code  just try this on you...
	 -->https://askubuntu.com/questions/61408/what-is-a-command-to-compile-and-run-c-programs

	 - vim is amazing! vim is a highly configurable text ...
	 -->https://askubuntu.com/questions/10998/what-developer-text-editors-are-available-for-ubuntu

	 - vim is a highly confi